### Spatial Analysis of UK Local Authority Districts (LADs)

In [ ]:
#importing libraries
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Load shapefiles
# LAD: Local Athority Districts 
# ITL1: UK regions (12 regions)

in_shp = r'data\Local_Authority_Districts_May_2024_Boundaries_UK_BGC_-5850961694214429102\LAD_MAY_2024_UK_BGC.shp'
lad_gdf = gpd.read_file(in_shp)

itl1_gdf = gpd.read_file(r'data\ITL1_JAN_2025_UK_BUC_3847796120714293736\ITL1_JAN_2025_UK_BUC.shp')

In [ ]:
#Check CRS
print('UK LAD CRS: ',lad_gdf.crs)
print('ITL1 CRS: ', itl1_gdf.crs)

In [ ]:
lad_gdf.info()

In [ ]:
itl1_gdf.info()

In [ ]:
itl1_gdf.head()

In [ ]:
lad_gdf.isnull().sum()

In [ ]:
#Clean data
lad_gdf.drop('LAD24NMW', axis=1, inplace=True)

In [ ]:
lad_gdf['geometry'] = lad_gdf.geometry.buffer(0)

In [ ]:
lad_gdf.head()

## Spatial join 
LADs to ITL1 regions

In [ ]:
lad_itl1 = gpd.sjoin(
    lad_gdf,
    itl1_gdf[['ITL125CD','ITL125NM','geometry']],
    how='left',
    predicate='intersects'
)

In [ ]:
lad_itl1

In [ ]:
# Reset index
lad_itl1.reset_index(drop=True, inplace=True)

### Feature Engineering

compute area, perimeter, compactness

In [ ]:
#area in km^2
lad_itl1['area_km2'] = lad_itl1.geometry.area/1e6

In [ ]:
# perimeter in km
lad_itl1['perimeter_km'] = lad_itl1.geometry.length/1000

In [ ]:
lad_itl1['compactness'] = 4 * np.pi * lad_itl1.geometry.area / (lad_itl1.geometry.length ** 2)

In [ ]:
# Urban/Rural proxy based on median area
median_area = lad_itl1['area_km2'].median()
lad_itl1['urban_rural_proxy'] = np.where(lad_itl1['area_km2'] < median_area, 'More Urban', 'More Rural')
lad_itl1['urban_rural_proxy']

In [ ]:
lad_itl1.head()

In [ ]:
#Save processed data
lad_itl1.to_file('lad_itl1.gpkg', driver='GPKG')

### Visualisation

In [ ]:
# LADs by ITL1 Region
fig, ax = plt.subplots(1, 1, figsize=(8, 10))
lad_itl1.plot(column='ITL125NM', edgecolor="black", linewidth=0.4, cmap='viridis', ax=ax)
ax.set_title("UK Local Authority Districts by ITL1 Region", fontsize=14)
ax.axis('off')
plt.show()

In [ ]:
# Urban/Rural proxy
lad_itl1.explore(column='urban_rural_proxy', tooltip=['LAD24NM', 'ITL125NM', 'area_km2', 'compactness'],\
                 legend=True, cmap='Spectral')

In [ ]:
lad_itl1.explore('LAD24NM', legend=False)

In [ ]:
# Compactness distribution
plt.figure(figsize=(8,6))
sns.histplot(lad_itl1['compactness'], bins=50, kde=True)
plt.title("Distribution of LAD Compactness")
plt.xlabel("Compactness")
plt.ylabel("Count")
plt.show()

Descriptive Statistics

In [ ]:
# Dropping rows with missing values in numeric columns for aggregation
clean_lad = lad_itl1.dropna(subset=['area_km2','perimeter_km','compactness'])

In [ ]:
itl_summary = clean_lad.groupby('ITL125NM')[['area_km2', 'perimeter_km', 'compactness']].agg(['mean','median','min','max'])
print("Summary by ITL1 Region")
print(itl_summary)

In [ ]:
# Summary by Urban/Rural
urban_rural_summary = clean_lad.groupby('urban_rural_proxy')[['area_km2','compactness']].agg(['mean','median'])
print("Summary by Urban/Rural Proxy")
print(urban_rural_summary)

In [ ]:
# Area by ITL1 region
plt.figure(figsize=(12,6))
sns.boxplot(data=clean_lad, x='ITL125NM', y='area_km2')
plt.xticks(rotation=45)
plt.title("Distribution of LAD Areas by ITL1 Region")
plt.ylabel("Area(km^2)")
plt.show()

The distribution of LAD areas varies widely across UK ITL1 regions.
London has the smallest LADs with relatively uniform sizes.
Scotland has the largest LADs on average, based on median area.
Regions like South West and North West show a broad range of district sizes, reflecting a mix of urban and rural areas.

In [ ]:
# Compactness Urban vs Rural
plt.figure(figsize=(8,6))
sns.boxplot(data=clean_lad, x='urban_rural_proxy', y='compactness', palette='Spectral')
plt.title("Compactness Comparison: Urban vs Rural LADs")
plt.show()

The boxplot compares the compactness values of Local Authority Districts classified as "More Urban" and "More Rural".
Urban LADs generally have higher compactness values compared to rural LADs,
suggesting they are more compact and regularly shaped.
Rural LADs tend to be less compact, likely due to larger and more irregular boundaries.

In [ ]:
# Conclusion 

print("Conclusion / Insights")
print(f"Total LADs: {len(clean_lad)}")
print(f"Median LAD area: {clean_lad['area_km2'].median():.2f} km^2")
print(f"Number of urban LADs: {len(clean_lad[clean_lad['urban_rural_proxy']=='More Urban'])}")
print(f"Number of rural LADs: {len(clean_lad[clean_lad['urban_rural_proxy']=='More Rural'])}")

print("\nInsights by ITL1 region:")
for region in clean_lad['ITL125NM'].unique():
    subset = clean_lad[clean_lad['ITL125NM']==region]
    print(f"{region}: median area = {subset['area_km2'].median():.2f} km^2, median compactness = {subset['compactness'].median():.2f}")

Interpretation

- Total LADs: 508, with 252 classified as "More Urban" and 256 as "More Rural".
- Largest LADs are in Scotland (~1120 km²) and Northern Ireland (~1251 km^2), smallest in London (~51 km^2).
- Urban LADs are smaller and more compact; rural LADs are larger and less compact.
- Compactness tends to be higher in urban areas and lower in large rural districts.
